In [5]:
from bs4 import BeautifulSoup as bs
import os
import pandas as pd
import numpy as np
from collections import Counter
import matplotlib.pyplot as plt
import glob
import seaborn as sns
import plotly.express as px
import descartes
import re
from datetime import datetime
import calmap
import geopandas
from pyspark import *
from pyspark.sql import *
from pyspark.sql.functions import *
import re
from pyspark.sql.types import ArrayType
from pyspark.sql.types import StringType
#import geoplot
#import shapefile as shp
import string

from sklearn.linear_model import LinearRegression, LogisticRegression
import geopandas
import descartes
import geoplot
import shapefile as shp
import warnings; warnings.simplefilter('ignore')

In [108]:
spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext
df = spark.read.option("header", "true").csv("data/cat_csv/*.csv")
df.count()
agora_vendors_df = df.select('vendor', 'date').distinct().toPandas()

In [200]:
SR_vendors_df = pd.read_csv("data/total.dat")
evo_vendors_df = pd.read_csv("data/evo_vendors.txt")
c9_vendors_df = pd.read_csv("data/c9_vendors.txt")

In [103]:
def cleanName(s):
    return str(s).translate(string.punctuation).lower()

In [201]:
agora_vendors_df['vendor']    = agora_vendors_df['vendor'].apply(cleanName)
SR_vendors_df['vendor']  = SR_vendors_df['vendor'].apply(cleanName)
evo_vendors_df['vendor'] = evo_vendors_df['vendor'].apply(cleanName)
c9_vendors_df['vendor']  = c9_vendors_df['vendor'].apply(cleanName)

In [202]:
agora_min_df = agora_vendors_df.groupby(by='vendor').min()
agora_max_df = agora_vendors_df.groupby(by='vendor').max()

In [212]:
evo_vendors_df['exact_Evo'] = 1
evo_min_df = evo_vendors_df.groupby(by='vendor').min()
evo_max_df = evo_vendors_df.groupby(by='vendor').max()

In [204]:
SR_vendors_df = pd.DataFrame.drop_duplicates(SR_vendors_df)
c9_vendors_df = pd.DataFrame.drop_duplicates(c9_vendors_df)

In [213]:
SR_vendors_df['exact_SR'] = 1
c9_vendors_df['exact_c9'] = 1

In [229]:
central_df = pd.DataFrame.drop_duplicates(agora_vendors_df.drop(columns=['date']))
central_df = central_df.set_index('vendor').join(agora_min_df).join(agora_max_df, lsuffix='_min', rsuffix='_max')

In [230]:
central_df = central_df.join(evo_min_df, how='left').join(evo_max_df, how='left', lsuffix='_evo_min', rsuffix='_evo_max').rename(columns={'exact_Evo_evo_max' : 'exact_Evo'}).drop(columns=['exact_Evo_evo_min'])

In [260]:
central_df = central_df.join(SR_vendors_df.set_index('vendor'), how='left').join(c9_vendors_df.set_index('vendor'), how='left')

ValueError: columns overlap but no suffix specified: Index(['exact_SR'], dtype='object')

In [262]:
central_df.reset_index(inplace=True)

In [268]:
import hashlib


In [575]:
def hamming_distance(chaine1, chaine2):
    if (len(chaine1) < 4) & (len(chaine1) != len(chaine2)):
        return 100
    return np.sum(c1 != c2 for c1, c2 in zip(chaine1, chaine2)) + np.abs((len(chaine1)-len(chaine2)))


def isThereAlmost(name, df):
    res = df['vendor'].apply(lambda x : hamming_distance(x, name))
    if len(res[res < 2]) > 0:
       # print(name)
       # print(res[res < 2])
        return 1
    else:
        return 0

In [576]:
central_df['hamming_SR'] = central_df['vendor'].apply(lambda x : isThereAlmost(x, SR_vendors_df))

In [577]:
central_df['hamming_evo'] = central_df['vendor'].apply(lambda x : isThereAlmost(x, evo_min_df.reset_index()))

KeyboardInterrupt: 

In [ ]:
central_df['hamming_c9'] = central_df['vendor'].apply(lambda x : isThereAlmost(x, c9_vendors_df))

In [334]:
survivor_all_mkt = central_df[(central_df['date_min'] < '2014-11-07') & (central_df['date_max'] > '2014-11-10')].dropna()
dead_all_mkt = central_df[(central_df['date_min'] < '2014-11-07') & (central_df['date_max'] < '2014-11-10')].dropna()

In [561]:
central_df.fillna(0, inplace=True)

In [578]:
central_df.to_csv('data/market_per_v_count.csv', index=None)

In [383]:
central_df['three_mkt'] = central_df['exact_Evo'] * central_df['exact_SR'] + central_df['exact_Evo'] * central_df['exact_c9'] + central_df['exact_SR'] *central_df['exact_c9']
central_df['three_mkt'] = central_df['three_mkt'].apply(lambda x:  1 if x > 1 else x)

In [ ]:
central_df['h_three_mkt'] = central_df['hamming_evo'] * central_df['hamming_SR'] + central_df['hamming_evo'] * central_df['hamming_c9'] + central_df['hamming_SR'] *central_df['hamming_c9']
central_df['h_three_mkt'] = central_df['h_three_mkt'].apply(lambda x:  1 if x > 1 else x)

In [ ]:
central_df['two_mkt'] = central_df['exact_Evo'] +  central_df['exact_c9'] + central_df['exact_SR']
central_df['two_mkt'] = central_df['two_mkt'].apply(lambda x:  1 if x > 1 else x)

central_df['h_two_mkt'] = central_df['hamming_evo'] +  central_df['hamming_c9'] + central_df['hamming_SR']
central_df['h_two_mkt'] = central_df['h_two_mkt'].apply(lambda x:  1 if x > 1 else x)

In [ ]:
central_df['all_mkt'] = central_df['exact_Evo'] * central_df['exact_c9'] * central_df['exact_SR']
central_df['h_all_mkt'] = central_df['hamming_evo'] * central_df['hamming_c9'] * central_df['hamming_SR']


In [ ]:
central_df['h_ag_only'] = 1 - central_df['h_two_mkt']

In [544]:
SR_vendors_df

,vendor,exact_SR
0,$220for28grams,1
1,*realdeal*,1
2,03welle,1
3,0ldamsterdam,1
4,1chitownmafia,1
...,...,...
46833,psybutiken,1
46915,sun-tzu,1
46926,swexyz,1
46932,t m n t,1


In [ ]:
central_df.sum()

In [444]:
agora_here_aft_df = central_df[central_df['date_max'] > '2014-12-10']
agora_here_bef_df = central_df[central_df['date_min'] < '2014-11-06']

## Population that surviced Onymous

In [ ]:
import matplotlib.pyplot as plt
import plotly.graph_objects as go
agora_surv_df = central_df[(central_df['date_max'] > '2015-01-10') &
                           (central_df['date_min'] < '2014-11-06')]
test = agora_surv_df.sum()
colors = ['gold', 'mediumturquoise', 'darkorange', 'lightgreen']

fig = go.Figure(data=[go.Pie(labels=['2', 'A', '3', '1'],
                             values=[test['h_two_mkt'] - test['h_three_mkt'] - test['h_all_mkt'], test['h_all_mkt'], test['h_three_mkt'], test['h_ag_only']])])
fig.update_traces(hoverinfo='label+percent', textinfo='value', textfont_size=20,
                  marker=dict(colors=colors, line=dict(color='#000000', width=2)))
fig.show()

# Data to 1


## Population arrived after Onymous

In [514]:
agora_aft_df = central_df[(central_df['date_m'] > '2014-11-06')]
test = agora_aft_df.sum()
# Data to 1
# Data to 1
colors = ['gold', 'mediumturquoise', 'darkorange', 'lightgreen']

fig = go.Figure(data=[go.Pie(labels=['2 Shops', '4 Shops', '3 Shops', '1 Shop'],
                             values=[test['two_mkt'] - test['three_mkt'] - test['all_mkt'], test['all_mkt'], test['three_mkt'], test['ag_only']])])
fig.update_traces(hoverinfo='label+percent', textinfo='value', textfont_size=20,
                  marker=dict(colors=colors, line=dict(color='#000000', width=2)))
fig.show()

# Data to 1


## Population that was there before Onymous

In [512]:
agora_bef_df = central_df[(central_df['date_min'] < '2014-11-06')]
test = agora_bef_df.sum()
# Data to 1
colors = ['gold', 'mediumturquoise', 'darkorange', 'lightgreen']

fig = go.Figure(data=[go.Pie(labels=['2 Shops', '4 Shops', '3 Shops', '1 Shop'],
                             values=[test['two_mkt'] - test['three_mkt'] - test['all_mkt'], test['all_mkt'], test['three_mkt'], test['ag_only']])])
fig.update_traces(hoverinfo='label+percent', textinfo='value', textfont_size=20,
                  marker=dict(colors=colors, line=dict(color='#000000', width=2)))
fig.show()

# Data to 1


## Population that died with Onymous

In [513]:
agora_dead_df = central_df[(central_df['date_max'] < '2014-11-06')]
test = agora_dead_df.sum()
# Data to 1
colors = ['gold', 'mediumturquoise', 'darkorange', 'lightgreen']

fig = go.Figure(data=[go.Pie(labels=['2 Shops', '4 Shops', '3 Shops', '1 Shop'],
                             values=[test['two_mkt'] - test['three_mkt'] - test['all_mkt'], test['all_mkt'], test['three_mkt'], test['ag_only']])])
fig.update_traces(hoverinfo='label+percent', textinfo='value', textfont_size=20,
                  marker=dict(colors=colors, line=dict(color='#000000', width=2)))
fig.show()

# Data to 1
